In [122]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup, element
from urllib.request import urlopen

In [ ]:

with urlopen('https://fbref.com/en/matches/f13e982e/Real-Betis-Girona-August-15-2024-La-Liga') as response:
    soup = BeautifulSoup(response, 'html.parser')
    tables = soup.find_all("table")
    table_df = {}
    for i, table in enumerate(tables):
        table_id = f"No name table #{i}" if table.get("id") is None else table.get("id")
        table_df[table_id] = pd.read_html(str(table))[0]

C:\Users\Huy's\AppData\Local\Temp\ipykernel_6956\982059425.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table_df[table_id] = pd.read_html(str(table))[0]
C:\Users\Huy's\AppData\Local\Temp\ipykernel_6956\982059425.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table_df[table_id] = pd.read_html(str(table))[0]
C:\Users\Huy's\AppData\Local\Temp\ipykernel_6956\982059425.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table_df[table_id] = pd.read_html(str(table))[0]
C:\Users\Huy's\AppData\Local\Temp\ipykernel_6956\982059425.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be rem

In [ ]:
# profile 
with urlopen('https://fbref.com/en/matches/f13e982e/Real-Betis-Girona-August-15-2024-La-Liga') as response:
    soup = BeautifulSoup(response, 'html.parser')
    tables = soup.find_all("table")
    table_df = {}
    for i, table in enumerate(tables):
        table_id = f"No name table #{i}" if table.get("id") is None else table.get("id")
        col_names = get_header(table)
        
        records = []
        columns = []
        for tr in table.findAll("tr"):
            ths = tr.findAll("th")
            if ths != []:
                for each in ths:
                    columns.append(each.text)
            else:
                trs = tr.findAll("td")
                record = []
                for each in trs:
                    try:
                        link = each.find('a')['href']
                        text = each.text
                        record.append(link)
                        record.append(text)
                    except:
                        text = each.text
                        record.append(text)
                records.append(record)

        columns.insert(1, 'Link')
        table_df[table_id] = pd.DataFrame(data=records)#, columns = columns)

In [ ]:
pd.read_html(tables[3].prettify())

C:\Users\Huy's\AppData\Local\Temp\ipykernel_6956\2755705327.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  pd.read_html(tables[3].prettify())


[   Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0  \
                Player                  #             Nation   
 0        Aitor Ruibal               24.0            es  ESP   
 1      Ezequiel Ávila                9.0            ar  ARG   
 2              Juanmi                7.0            es  ESP   
 3     Abde Ezzalzouli               10.0            ma  MAR   
 4       Pablo Fornals               18.0            es  ESP   
 5               Rodri               17.0            es  ESP   
 6         Nabil Fekir                8.0            fr  FRA   
 7           Marc Roca               21.0            es  ESP   
 8      Sergi Altimira               16.0            es  ESP   
 9    William Carvalho               14.0            pt  POR   
 10     Johnny Cardoso                4.0            us  USA   
 11     Romain Perraud               15.0            fr  FRA   
 12     Diego Llorente                3.0            es  ESP   
 13        Marc Bartra                5.

In [ ]:
type(tables[3])

bs4.element.Tag

In [ ]:
table_df.keys()

dict_keys(['No name table #0', 'No name table #1', 'No name table #2', 'stats_fc536746_summary', 'stats_fc536746_passing', 'stats_fc536746_passing_types', 'stats_fc536746_defense', 'stats_fc536746_possession', 'stats_fc536746_misc', 'keeper_stats_fc536746', 'stats_9024a00a_summary', 'stats_9024a00a_passing', 'stats_9024a00a_passing_types', 'stats_9024a00a_defense', 'stats_9024a00a_possession', 'stats_9024a00a_misc', 'keeper_stats_9024a00a', 'shots_all', 'shots_fc536746', 'shots_9024a00a'])

In [ ]:
table_df["stats_fc536746_summary"]

""


In [65]:
tables = soup.find_all("table")
type(tables[0])

bs4.element.Tag

In [170]:
def tbl_has_head_group(table: element.Tag) -> bool:
    """
    Check if table has thead element.
    """
    return bool(table.find("thead"))

def tr_tag_header_to_list(tr: element.Tag, col_identifier: str) -> list:
    """
    This captures all header columns in a tag and return a list corresponding to how many column each header spans.
    """
    header_list = []
    for col in tr.find_all("th"):
        col_text = col.get(col_identifier, "header")
        col_span = int(col.get("colspan", 1))
        header_list.extend([col_text]*col_span)
    return header_list

def get_header(table: element.Tag):
    if tbl_has_head_group(table):
        header_name = {}
        header_name_list = []
        head_lvls = table.find("thead").find_all("tr")
        
        for i, head_lvl in enumerate(head_lvls):
            header_name[i] = tr_tag_header_to_list(head_lvl, col_identifier="data-stat")
        
        # iterate each column over the longest list in the dict of header rows
        header_name_list = []
        for col in range(max([len(i) for i in header_name.values()])):
            header_final_name = ""
            for row in range(len(header_name)):
                delimiter = "" if header_final_name == "" else "_"
                header_final_name = header_final_name + delimiter + header_name[row][col]
            header_name_list.append(header_final_name)
        return header_name
    raise ValueError("Table element had no thead tag")
    

In [174]:
print(get_header(tables[3])[1])

['player', 'shirtnumber', 'nationality', 'position', 'age', 'minutes', 'goals', 'assists', 'pens_made', 'pens_att', 'shots', 'shots_on_target', 'cards_yellow', 'cards_red', 'touches', 'tackles', 'interceptions', 'blocks', 'xg', 'npxg', 'xg_assist', 'sca', 'gca', 'passes_completed', 'passes', 'passes_pct', 'progressive_passes', 'carries', 'progressive_carries', 'take_ons', 'take_ons_won']


# Use pydantic and bs4 to scrape and validate data for each table

In [229]:
from pydantic import BaseModel, RootModel
from decimal import Decimal

class MatchPlayerSummaryRow(BaseModel):
    """
    Model to validate each row of data scraped from summary table in fbref.
    """
    player_id: str
    player_name: str
    position: str
    # age: str
    minutes: int
    goals: int | None
    assists: int | None
    pens_made: int | None
    pens_att: int | None
    shots: int | None
    shots_on_target: int | None
    cards_yellow: int | None
    cards_red: int | None
    touches: int | None
    tackles: int | None
    interceptions: int | None
    blocks: int | None
    xg: Decimal | None
    npxg: Decimal | None
    xg_assist: Decimal | None
    sca: int | None
    gca: int | None
    passes_completed: int | None
    passes: int | None
    passes_pct: Decimal | None
    progressive_passes: int | None
    carries: int | None
    progressive_carries: int | None
    take_ons: int | None
    take_ons_won: int | None

In [215]:
def tbl_has_body_group(table: element.Tag) -> bool:
    """
    Check if table has thead element.
    """
    return bool(table.find("tbody"))

def get_all_player_data(table:element.Tag, pydantic_model: BaseModel) -> pd.DataFrame:
    data = []
    if tbl_has_body_group(table):
        for row in table.select('tbody tr'):
            row_dict = {}
            header = row.select('th[data-stat="player"]')[0]
            row_dict["player_id"] = header.get("data-append-csv")
            row_dict["player_name"] = header.get("csk")
            body_fields = {key for key in pydantic_model.model_fields.keys() if key not in ["player_id", "player_name"]}
            for field in body_fields:
                #if field not in ["player_id", "player_name"]:
                row_dict[field] = row.select(f'td[data-stat="{field}"]')[0].text
            player_row = pydantic_model(**row_dict)
            data.append(player_row)
        df = pd.DataFrame([s.__dict__ for s in data])
    return df

In [204]:
MatchPlayerSummaryRow.model_fields_set

In [230]:
get_all_player_data(tables[3], MatchPlayerSummaryRow)

,player_id,player_name,position,minutes,goals,assists,pens_made,pens_att,shots,shots_on_target,...,sca,gca,passes_completed,passes,passes_pct,progressive_passes,carries,progressive_carries,take_ons,take_ons_won
0,a55d8df5,Ruibal Aitor,FW,69,0,0,0,0,3,1,...,4,0,11,16,68.8,2,16,2,0,0
1,253a06b9,Ávila Ezequiel,FW,21,0,0,0,0,1,0,...,2,0,5,5,100.0,0,4,0,0,0
2,84399660,Juanmi,LW,58,0,0,0,0,1,0,...,1,0,8,10,80.0,1,11,1,2,1
3,bed68338,Ezzalzouli Abde,LW,32,0,0,0,0,0,0,...,0,0,8,9,88.9,2,10,2,4,3
4,82927de4,Fornals Pablo,RW,58,0,0,0,0,2,0,...,6,0,15,21,71.4,5,19,1,1,0
5,85bed4ee,Rodri,RW,32,0,0,0,0,1,1,...,3,0,11,15,73.3,2,19,3,2,0
6,bece776f,Fekir Nabil,AM,90,0,1,0,0,2,0,...,9,1,29,34,85.3,5,30,5,2,0
7,da27e268,Roca Marc,DM,76,0,0,0,0,0,0,...,1,0,14,18,77.8,2,13,0,0,0
8,94acdd4e,Altimira Sergi,DM,14,0,0,0,0,0,0,...,2,0,18,20,90.0,3,16,0,0,0
9,dee86451,Carvalho William,DM,76,0,0,0,0,2,1,...,3,0,31,35,88.6,2,31,1,1,0


In [ ]:
print(get_header(tables[4])[1])

['player', 'shirtnumber', 'nationality', 'position', 'age', 'minutes', 'passes_completed', 'passes', 'passes_pct', 'passes_total_distance', 'passes_progressive_distance', 'passes_completed_short', 'passes_short', 'passes_pct_short', 'passes_completed_medium', 'passes_medium', 'passes_pct_medium', 'passes_completed_long', 'passes_long', 'passes_pct_long', 'assists', 'xg_assist', 'pass_xa', 'assisted_shots', 'passes_into_final_third', 'passes_into_penalty_area', 'crosses_into_penalty_area', 'progressive_passes']


In [225]:
from pydantic.validators import str_validator
from typing import Optional, Union

def empty_to_none(v: str) -> Optional[str]:
    if v == '':
        return None
    return v

class EmptyStrToNone(str):
    @classmethod
    def __get_validators__(cls):
        yield str_validator
        yield empty_to_none

class MatchPlayerPassRow(BaseModel):
    player_id: str
    player_name: str
    position: str
    age: str
    minutes: int
    passes_completed: int | None
    passes: int | None
    passes_pct: Decimal | None
    passes_total_distance: int | None
    passes_progressive_distance: int | None
    passes_completed_short: int | None
    passes_short: int | None
    passes_pct_short: Decimal | None
    passes_completed_medium: int | None
    passes_medium: int | None
    passes_pct_medium: Decimal | None
    passes_completed_long: int | None
    passes_long: int | None
    passes_pct_long: Decimal | None
    assists: int | None
    xg_assist: Decimal | None
    pass_xa: Decimal | None
    assisted_shots: int | None
    passes_into_final_third: int | None
    passes_into_penalty_area: int | None
    crosses_into_penalty_area: int | None
    progressive_passes: int | None

In [226]:
get_all_player_data(tables[4], MatchPlayerPassRow)

ValidationError: 1 validation error for MatchPlayerPassRow
passes_pct_long
  Input should be a valid decimal [type=decimal_parsing, input_value='', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/decimal_parsing